<a href="https://colab.research.google.com/github/chongzicbo/Dive-into-Deep-Learning-tf.keras/blob/master/6.7.%20%E9%97%A8%E6%8E%A7%E5%BE%AA%E7%8E%AF%E5%8D%95%E5%85%83%EF%BC%88GRU%EF%BC%89.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##6.7. 门控循环单元（GRU）
上一节介绍了循环神经网络中的梯度计算方法。我们发现，当时间步数较大或者时间步较小时，循环神经网络的梯度较容易出现衰减或爆炸。虽然裁剪梯度可以应对梯度爆炸，但无法解决梯度衰减的问题。通常由于这个原因，循环神经网络在实际中较难捕捉时间序列中时间步距离较大的依赖关系。

门控循环神经网络（gated recurrent neural network）的提出，正是为了更好地捕捉时间序列中时间步距离较大的依赖关系。它通过可以学习的门来控制信息的流动。其中，门控循环单元（gated recurrent unit，GRU）是一种常用的门控循环神经网络 [1, 2]。另一种常用的门控循环神经网络则将在下一节中介绍。

###6.7.1. 门控循环单元
下面将介绍门控循环单元的设计。它引入了重置门（reset gate）和更新门（update gate）的概念，从而修改了循环神经网络中隐藏状态的计算方式。

####6.7.1.1. 重置门和更新门
如图6.4所示，门控循环单元中的重置门和更新门的输入均为当前时间步输入 $X_t$ 与上一时间步隐藏状态 $H_{t−1}$ ，输出由激活函数为sigmoid函数的全连接层计算得到。

<img src="https://zh.gluon.ai/_images/gru_1.svg" width="500"/>

<center>图 6.4 门控循环单元中重置门和更新门的计算</center>

具体来说，假设隐藏单元个数为 $h$ ，给定时间步 $t$的小批量输入$\boldsymbol{X}_t \in \mathbb{R}^{n \times d}$（样本数为 $n$ ，输入个数为 $d$ ）和上一时间步隐藏状态$\boldsymbol{H}_{t-1} \in \mathbb{R}^{n \times h}$。重置门 $\boldsymbol{R}_t \in \mathbb{R}^{n \times h}$和更新门$\boldsymbol{Z}_t \in \mathbb{R}^{n \times h}$的计算如下：

$$\begin{split}\begin{aligned}
\boldsymbol{R}_t = \sigma(\boldsymbol{X}_t \boldsymbol{W}_{xr} + \boldsymbol{H}_{t-1} \boldsymbol{W}_{hr} + \boldsymbol{b}_r),\\
\boldsymbol{Z}_t = \sigma(\boldsymbol{X}_t \boldsymbol{W}_{xz} + \boldsymbol{H}_{t-1} \boldsymbol{W}_{hz} + \boldsymbol{b}_z),
\end{aligned}\end{split}$$
其中 $\boldsymbol{W}_{xr}, \boldsymbol{W}_{xz} \in \mathbb{R}^{d \times h}$和$\boldsymbol{W}_{hr}, \boldsymbol{W}_{hz} \in \mathbb{R}^{h \times h}$是权重参数，$\boldsymbol{b}_r, \boldsymbol{b}_z \in \mathbb{R}^{1 \times h}$是偏差参数。“多层感知机”一节中介绍过，sigmoid函数可以将元素的值变换到0和1之间。因此，重置门$\boldsymbol{R}_t$和更新门$\boldsymbol{Z}_t$中每个元素的值域都是 [0,1] 。
####6.7.1.2. 候选隐藏状态
接下来，门控循环单元将计算候选隐藏状态来辅助稍后的隐藏状态计算。如图6.5所示，我们将当前时间步重置门的输出与上一时间步隐藏状态做按元素乘法（符号为 $\odot$ ）。如果重置门中元素值接近0，那么意味着重置对应隐藏状态元素为0，即丢弃上一时间步的隐藏状态。如果元素值接近1，那么表示保留上一时间步的隐藏状态。然后，将按元素乘法的结果与当前时间步的输入连结，再通过含激活函数tanh的全连接层计算出候选隐藏状态，其所有元素的值域为$[-1, 1]$ 。
<img src="https://zh.gluon.ai/_images/gru_2.svg" width="500"/>

<center>图 6.5 门控循环单元中候选隐藏状态的计算。这里的 $\odot$是按元素乘法</center>

具体来说，时间步 $t$的候选隐藏状态$\tilde{\boldsymbol{H}}_t \in \mathbb{R}^{n \times h}$的计算为
$$
\tilde{\boldsymbol{H}}_t = \text{tanh}(\boldsymbol{X}_t \boldsymbol{W}_{xh} + \left(\boldsymbol{R}_t \odot \boldsymbol{H}_{t-1}\right) \boldsymbol{W}_{hh} + \boldsymbol{b}_h),
$$
其中 $\boldsymbol{W}_{xh} \in \mathbb{R}^{d \times h}$和$\boldsymbol{W}_{hh} \in \mathbb{R}^{h \times h}$是权重参数，$\boldsymbol{b}_h \in \mathbb{R}^{1 \times h}$是偏差参数。从上面这个公式可以看出，重置门控制了上一时间步的隐藏状态如何流入当前时间步的候选隐藏状态。而上一时间步的隐藏状态可能包含了时间序列截至上一时间步的全部历史信息。因此，重置门可以用来丢弃与预测无关的历史信息。

####6.7.1.3. 隐藏状态
最后，时间步 $t$ 的隐藏状态$\boldsymbol{H}_t \in \mathbb{R}^{n \times h}$的计算使用当前时间步的更新门 $\boldsymbol{Z}_t$来对上一时间步的隐藏状态 $H_{t−1}$ 和当前时间步的候选隐藏状态 

$$
\boldsymbol{H}_t = \boldsymbol{Z}_t \odot \boldsymbol{H}_{t-1}  + (1 - \boldsymbol{Z}_t) \odot \tilde{\boldsymbol{H}}_t.
$$

<img src="https://zh.gluon.ai/_images/gru_3.svg" width="500"/>

<center>图 6.6 门控循环单元中隐藏状态的计算。这里的$\odot$是按元素乘法</center>

值得注意的是，更新门可以控制隐藏状态应该如何被包含当前时间步信息的候选隐藏状态所更新，如图6.6所示。假设更新门在时间步$t'$到$t$(t' < t)之间一直近似1。那么，在时间步$t'$到$t$之间的输入信息几乎没有流入时间步 $t$ 的隐藏状态 $H_t$ 。实际上，这可以看作是较早时刻的隐藏状态$\boldsymbol{H}_{t'-1}$一直通过时间保存并传递至当前时间步 $t$ 。这个设计可以应对循环神经网络中的梯度衰减问题，并更好地捕捉时间序列中时间步距离较大的依赖关系。

我们对门控循环单元的设计稍作总结：

* 重置门有助于捕捉时间序列里短期的依赖关系；
* 更新门有助于捕捉时间序列里长期的依赖关系。

###6.7.2. 读取数据集
为了实现并展示门控循环单元，下面依然使用周杰伦歌词数据集来训练模型作词。这里除门控循环单元以外的实现已在“循环神经网络”一节中介绍过。以下为读取数据集部分。

In [0]:
%matplotlib inline
import math
import tensorflow as tf
import numpy as np
from IPython import display
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import losses
from tensorflow.data import Dataset
import time
import random
import zipfile

In [0]:
tf.enable_eager_execution()

In [0]:
def load_data_jay_lyrics():
  from google.colab import drive
  drive.mount('/content/drive')
  with zipfile.ZipFile('/content/drive/My Drive/data/d2l-zh-tensoflow/jaychou_lyrics.txt.zip')as zin:
    with zin.open('jaychou_lyrics.txt') as f:
      corpus_chars=f.read().decode('utf-8')
  corpus_chars=corpus_chars.replace('\n',' ').replace('\r',' ')
  corpus_chars=corpus_chars[0:10000]
  idx_to_char=list(set(corpus_chars))
  char_to_idx=dict([(char,i) for i,char in enumerate(idx_to_char)])
  vocab_size=len(char_to_idx)
  corpus_indices=[char_to_idx[char] for char in corpus_chars]
  return corpus_indices,char_to_idx,idx_to_char,vocab_size

(corpus_indices,char_to_idx,idx_to_char,vocab_size)=load_data_jay_lyrics() 

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


###6.7.3. 从零开始实现
我们先介绍如何从零开始实现门控循环单元。

####6.7.3.1. 初始化模型参数
下面的代码对模型参数进行初始化。超参数num_hiddens定义了隐藏单元的个数。

In [0]:
num_inputs,num_hiddens,num_outputs=vocab_size,256,vocab_size
def get_params():
  def _one(shape):
    return tf.Variable(tf.random.normal(stddev=0.01,shape=shape))

  def _three():
    return (_one((num_inputs,num_hiddens)),
        _one((num_hiddens,num_hiddens)),
        tf.Variable(tf.zeros(num_hiddens)))
    
  W_xz,W_hz,b_z=_three() #更新们参数
  W_xr,W_hr,b_r=_three() #重置门参数
  W_xh,W_hh,b_h=_three() #候选隐藏状态参数

  #输出层参数
  W_hq=_one((num_hiddens,num_outputs))
  b_q=tf.Variable(tf.zeros(num_outputs))

  params = [W_xz, W_hz, b_z, W_xr, W_hr, b_r, W_xh, W_hh, b_h, W_hq, b_q]
  return params

####6.7.3.2. 定义模型
下面的代码定义隐藏状态初始化函数init_gru_state。同“循环神经网络的从零开始实现”一节中定义的init_rnn_state函数一样，它返回由一个形状为(批量大小, 隐藏单元个数)的值为0的Tensor组成的元组。

In [0]:
def init_gru_state(batch_size,num_hiddens):
  return (tf.zeros(shape=(batch_size,num_hiddens)),)

下面根据门控循环单元的计算表达式定义模型。

In [0]:
def gru(inputs,state,params):
  W_xz, W_hz, b_z, W_xr, W_hr, b_r, W_xh, W_hh, b_h, W_hq, b_q=params
  H,=state
  outputs=[]
  for X in inputs:
    Z=tf.sigmoid(tf.matmul(X,W_xz)+tf.matmul(H,W_hz)+b_z) #更新门
    R=tf.sigmoid(tf.matmul(X,W_xr)+tf.matmul(H,W_hr)+b_r) #重置门
    H_tilda=tf.tanh(tf.matmul(X,W_xh)+tf.matmul(R*H,W_hh)+b_h)
    H=Z*H+(1-Z)*H_tilda
    Y=tf.matmul(H,W_hq)+b_q
    outputs.append(Y)
  return outputs,(H,)

####6.7.3.3. 训练模型并创作歌词
我们在训练模型时只使用相邻采样。设置好超参数后，我们将训练模型并根据前缀“分开”和“不分开”分别创作长度为50个字符的一段歌词。

In [0]:
num_epochs, num_steps, batch_size, lr, clipping_theta = 160, 35, 32, 1e2, 1e-2
pred_period, pred_len, prefixes = 40, 50, ['分开', '不分开']

In [0]:
def to_onehot(X,size):
  return [tf.one_hot(x,size) for x in tf.transpose(X)]

In [0]:
def predict_rnn(prefix,num_chars,rnn,params,init_rnn_state,num_hiddens,vocab_size,idx_to_char,char_to_idx):
  state=init_rnn_state(1,num_hiddens)
  output=[char_to_idx[prefix[0]]]
  for t in range(num_chars+len(prefix)-1):
    #将上一时间步的输出作为当前时间步的输入
    X=to_onehot(tf.reshape(tf.constant([output[-1]]),shape=(1,1)),vocab_size)
    # print(X[0].shape)
    #计算输出和更新隐藏状态
    (Y,state)=rnn(X,state,params)
    #下一个时间步的输入是prefix里的字符或者当前的最佳预测字符
    if t<len(prefix)-1:
      output.append(char_to_idx[prefix[t+1]])
    else:
      output.append(tf.argmax(Y[0],axis=1).numpy()[0])
  return ''.join([idx_to_char[i] for i in output])

In [0]:
def data_iter_random(corpus_indices,batch_size,num_steps):
  #减1是因为输出的索引是相应输入的索引加1
  num_examples=(len(corpus_indices)-1)//num_steps
  epoch_size=num_examples//batch_size
  example_indices=list(range(num_examples))
  random.shuffle(example_indices)

  #返回从pos开始的长为num_steps的序列
  def _data(pos):
    return corpus_indices[pos:pos+num_steps]

  for i in range(epoch_size):
    #每次读取batch_size个随机样本
    i=i*batch_size
    batch_indices=example_indices[i:i+batch_size]
    X=[_data(j*num_steps) for j in batch_indices]
    Y=[_data(j*num_steps+1) for j in batch_indices]
    yield tf.constant(X),tf.constant(Y)

def data_iter_consecutive(corpus_indices,batch_size,num_steps):
  corpus_indices=tf.constant(corpus_indices)
  data_len=len(corpus_indices)
  batch_len=data_len//batch_size
  indices=tf.reshape(corpus_indices[0:batch_size*batch_len],shape=(batch_size,batch_len))
  epoch_size=(batch_len-1) // num_steps
  for i in range(epoch_size):
    i=i*num_steps
    X=indices[:,i:i+num_steps]
    Y=indices[:,i+1:i+num_steps+1]
    yield X,Y    

In [0]:
def sgd(params,l,t,lr,batch_size,theta):
  norm=tf.constant([0],dtype=tf.float32)
  for param in params:
    dl_dp=t.gradient(l,param)
    if dl_dp is None:
      print(param,dl_dp)
    norm+=tf.reduce_sum((dl_dp**2))
  norm=tf.sqrt(norm).numpy()
  if norm>theta:
    for param in params:
      dl_dp=t.gradient(l,param) #求梯度
      dl_dp=tf.assign(tf.Variable(dl_dp),dl_dp*theta/norm) #裁剪梯度
      param.assign_sub(lr*dl_dp/batch_size) #更新梯度

In [0]:
def train_and_predict_rnn(rnn,get_params,init_rnn_state,num_hiddens,vocab_size,corpus_indices,idx_to_char,char_to_idx,is_random_iter,num_epochs,
                          num_steps,lr,clipping_theta,batch_size,pred_period,pred_len,prefixes):
  if is_random_iter:
    data_iter_fn=data_iter_random
  else:
    data_iter_fn=data_iter_consecutive

  params=get_params()
  loss=losses.SparseCategoricalCrossentropy(from_logits=True)

  for epoch in range(num_epochs):
    if not is_random_iter:#如果使用相邻采样，在epoch开始时初始化隐藏状态
      state=init_rnn_state(batch_size,num_hiddens)
    l_sum,n,start=0.0,0,time.time()
    data_iter=data_iter_fn(corpus_indices,batch_size,num_steps)
    for X,Y in data_iter:
      if is_random_iter:#如果使用相邻采样，在每个小批量更新前初始化隐藏状态
        state=init_rnn_state(batch_size,num_hiddens)
      else:#否则需要使用detach函数从
        state=tf.stop_gradient(state) #停止计算该张量的梯度
      with tf.GradientTape(persistent=True) as t:
        t.watch(params)
        inputs=to_onehot(X,vocab_size)
        # outputs有num_steps个形状为(batch_size, vocab_size)的矩阵
        (outputs,state)=rnn(inputs,state,params)
        # 拼接之后形状为(num_steps * batch_size, vocab_size)
        outputs=tf.concat(values=[*outputs],axis=0)
        # Y的形状是(batch_size, num_steps)，转置后再变成长度为
        # batch * num_steps 的向量，这样跟输出的行一一对应
        y=tf.reshape(tf.transpose(Y),shape=(-1,))
        #使用交叉熵损失计算平均分类误差
        l=tf.reduce_mean(loss(y,outputs))
      sgd(params,l,t,lr,1,clipping_theta) #因为误差已经取过均值了,所以batch_size为1
      l_sum+=l.numpy()*y.numpy().shape[0]
      n+=y.numpy().shape[0]
    if(epoch +1)%10==0:
      print('epoch %d, perplexity %f, time %.2f sec' % (
                epoch + 1, l_sum / n, time.time() - start))
      for prefix in prefixes:
          print(' -', predict_rnn(
              prefix, pred_len, rnn, params, init_rnn_state,
              num_hiddens, vocab_size, idx_to_char, char_to_idx))

In [0]:
train_and_predict_rnn(gru,get_params,init_gru_state,num_hiddens,vocab_size,corpus_indices,idx_to_char,char_to_idx,False,num_epochs,num_steps,lr,clipping_theta,batch_size,pred_period,pred_len,prefixes)

epoch 10, perplexity 5.705591, time 9.95 sec
 - 分开 我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我
 - 不分开我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我
epoch 20, perplexity 5.573065, time 10.10 sec
 - 分开 我我我的 我我我的 我我我的 我我我的 我我我的 我我我的 我我我的 我我我的 我我我的 我我我的
 - 不分开 我我我的 我我我的 我我我的 我我我的 我我我的 我我我的 我我我的 我我我的 我我我的 我我我的
epoch 30, perplexity 5.339191, time 10.29 sec
 - 分开 我不不 我不你的我 我不不 我不你的我 我不不 我不你的我 我不不 我不你的我 我不不 我不你的我
 - 不分开 我不不的我 我不不 我不你的我 我不不 我不你的我 我不不 我不你的我 我不不 我不你的我 我不不
epoch 40, perplexity 5.010512, time 10.20 sec
 - 分开 我想你的让我爱爱女 爱爱女人 我想你的可爱女人 爱爱的让我爱女女 爱爱女人 我想你的可爱女人 爱爱
 - 不分开 我想你的让我爱爱女 爱爱女人 我想你的可爱女人 爱爱的让我爱女女 爱爱女人 我想你的可爱女人 爱爱
epoch 50, perplexity 4.675587, time 9.81 sec
 - 分开 我想你的爱爱人 爱爱女人 我想想你的可爱女人 坏坏的让我疯狂的可爱女人 坏坏的让我疯狂的可爱女人 
 - 不分开 我想你的爱爱女人  爱有你的爱爱人 爱爱女人 我不要你的可爱女人 坏坏的让我疯狂的可爱女人 坏坏的
epoch 60, perplexity 4.306026, time 9.73 sec
 - 分开 我想要你的爱爱  爱有你的爱写 一场人 我不要 我不要 我不要 我不要 我不要 我不要 我不要 我
 - 不分开 我想你的爱爱 一场人 我不要 我不要 我不要 我不要 我不要 我不要 我不要 我不要 我不要 我不
epoch 70, perplexity 3.936894, time 9.90 sec
 - 分

###6.7.4. 小结
* 门控循环神经网络可以更好地捕捉时间序列中时间步距离较大的依赖关系。
* 门控循环单元引入了门的概念，从而修改了循环神经网络中隐藏状态的计算方式。它包括重置门、更新门、候选隐藏状态和隐藏状态。
* 重置门有助于捕捉时间序列里短期的依赖关系。
* 更新门有助于捕捉时间序列里长期的依赖关系。